# Passband Demo

A demonstartion of using the `Passbands` class to calculate normalized in-band flux densities.

This code and notebook reference equations in *"On the Choice of LSST Flux Units"* by Željko Ivezić and the LSST Project Science Team, which may be found [here](https://docushare.lsst.org/docushare/dsweb/Get/Document-27758).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tdastro.astro_utils.passbands import Passbands
from tdastro.sources.spline_model import SplineModel

## Set up LSST passbands

These values are unnormalized; we'll need to apply phi from eq 8 of the LSST Flux paper.

In [ ]:
passbands = Passbands()
passbands._get_data_path()
passbands.load_all_transmission_tables()

In [ ]:
def plot_single_transmission_table(band_name: str) -> None:
    """Plot a single transmission table."""
    plt.title(f"{band_name}-band Transmission Table")
    plt.xlabel("Wavelength (Angstroms)")
    plt.ylabel("Transmission")
    plt.plot(passbands.transmission_tables[band_name][:, 0], passbands.transmission_tables[band_name][:, 1])
    plt.show()


plot_single_transmission_table("u")

In [ ]:
def plot_transmission_tables() -> None:
    """Plot all transmission tables."""
    plt.title("Transmission Tables")
    plt.xlabel("Wavelength (Angstroms)")
    plt.ylabel("Transmission")
    plt.legend(passbands.bands)
    for band_id in passbands.bands:
        plt.plot(passbands.transmission_tables[band_id][:, 0], passbands.transmission_tables[band_id][:, 1])
    plt.show()


plot_transmission_tables()

## Get normalized passband

In [ ]:
passbands.calculate_normalized_system_response_tables()

In [ ]:
def plot_single_normalized_system_response_table(band_name: str) -> None:
    """Plot a single normalized system response table."""
    plt.title(f"{band_name}-band Normalized System Response Table")
    plt.xlabel("Wavelength (Angstroms)")
    plt.ylabel("Normalized System Response (inverse Angstroms I guess? check this whole axis label)")
    plt.plot(
        passbands.normalized_system_response_tables[band_name][:, 0],
        passbands.normalized_system_response_tables[band_name][:, 1],
    )
    plt.show()


plot_single_normalized_system_response_table("u")

In [ ]:
def plot_normalized_system_response_tables() -> None:
    """Plot all normalized system response tables."""
    plt.title("Normalized System Response Tables")
    plt.xlabel("Wavelength (Angstroms)")
    plt.ylabel("Normalized System Response (inverse Angstroms I guess? check this whole axis label)")
    plt.legend(passbands.bands)
    for band_id in passbands.bands:
        plt.plot(
            passbands.normalized_system_response_tables[band_id][:, 0],
            passbands.normalized_system_response_tables[band_id][:, 1],
        )
    plt.show()


plot_normalized_system_response_tables()

## Set up a source

Note that our transmission tables are to the closest integer; eg, in 'u', all integers in [3200, 4085].

Models that follow a different grid might need to be put through a SplineModel to match this grid.

In [ ]:
input_times = np.array([1001.0, 1002.0, 1003.0])
input_wavelengths = np.linspace(2000.0, 6000.0, 3)
input_fluxes = np.array([[1.0, 5.0, 1.0], [5.0, 10.0, 5.0], [1.0, 5.0, 1.0]])

spline_model = SplineModel(input_times, input_wavelengths, input_fluxes, time_degree=2, wave_degree=2)

times = np.linspace(1000.0, 1006.0, 48)
wavelengths = np.linspace(2000.0, 8000.0, 16)
fluxes = spline_model.evaluate(times, wavelengths)

In [ ]:
def plot_flux_spectrogram():
    """Plot a spectrogram to visualize the fluxes."""
    plt.figure(figsize=(12, 3))  # Increase the width (14) and keep the height (6)
    plt.imshow(fluxes.T, cmap="plasma", interpolation="nearest")

    # Add title, axis labels, and correct ticks
    plt.title("Flux Spectrogram (units?)")
    plt.xlabel("Time (idk, days maybe)")
    plt.ylabel("Wavelength (Angstroms)")
    plt.xticks(np.arange(len(times))[::4], [f"{round(time)}" for time in times][::4])
    plt.yticks(np.arange(len(wavelengths))[::2], [f"{round(wave)}" for wave in wavelengths][::2])

    # Add flux labels
    for (j, i), label in np.ndenumerate(fluxes.T):
        if i % 2 == 1:
            plt.text(i, j, round(label, 1), ha="center", va="center", size=8)

    plt.show()


plot_flux_spectrogram()

## Calculate in-band flux densities

In [ ]:
per_passband_normalized_fluxes = passbands.get_all_in_band_fluxes(spline_model, times)

In [ ]:
# Print in-band normalized fluxes (abridged-we only print every 4th value, for space)
np.set_printoptions(precision=2)
print(per_passband_normalized_fluxes[::4])

In [ ]:
def plot_passband_normalized_lightcurve():
    """Plot the passband-normalized lightcurve."""
    fig, ax = plt.subplots()
    ax.set_title("Passband-Normalized Lightcurve")
    ax.set_xlabel("Time (days)")
    ax.set_ylabel("Flux (units?)")

    offset = 0.015  # Optionally add a small offset for the overlapping bandss
    for i, band in enumerate(passbands.bands):
        ax.plot(times + offset * i, per_passband_normalized_fluxes[:, i] + offset * i, marker="o", label=band)

    ax.legend()
    plt.show()


plot_passband_normalized_lightcurve()